In [2]:
import sys, importlib
sys.path.append(r"C:\Users\Artem\Desktop\vs code project\chess_forecast\src")
importlib.reload(importlib.import_module('features'))
importlib.reload(importlib.import_module('move'))
importlib.reload(importlib.import_module('mymodel'))
importlib.reload(importlib.import_module('training'))

import features, move, mymodel, training
import numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [3]:
ALL_MOVES = move.generate_all_possible_moves()
MOVE_TO_INDEX = {m.uci(): i for i, m in enumerate(ALL_MOVES)}
INDEX_TO_MOVE = {i: m for i, m in enumerate(ALL_MOVES)}
len(MOVE_TO_INDEX)

4208

In [4]:
path = '../fens_training_set.csv'
df = pd.read_csv(path)
df = df.drop(232329, axis=0)
df = df[:10000]

In [8]:
#Разделение на train/val
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = mymodel.ChessDataset(train_df, ALL_MOVES, MOVE_TO_INDEX)
test_dataset = mymodel.ChessDataset(test_df, ALL_MOVES, MOVE_TO_INDEX)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=0)

In [ ]:
# Модель
num_epochs = 5
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = mymodel.ChessMovePredictor(num_moves=len(ALL_MOVES)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.01, 
    epochs=num_epochs, steps_per_epoch=len(train_loader),
    pct_start=0.1)
criterion = mymodel.ChessMoveLoss()

In [10]:
#Обучение
#Train loss: 1.7564, acc: 0.4661
#Validation: Loss: 2.2240, Acc@1: 0.361, Acc@3: 0.598, Acc@5: 0.704
best_val_acc = 0
for epoch in range(1, num_epochs + 1):
    print(f"\nEPOCH {epoch}/{num_epochs}")
    train_loss, train_acc = training.train_epoch(model, train_loader, optimizer, criterion, device,scheduler)
    val_metrics = training.validate_epoch(model, test_loader, criterion, device)
    val_acc = val_metrics['acc1']
    print(f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    print(f"Validation: Loss: {val_metrics['loss']:.4f}, "
      f"Acc@1: {val_metrics['acc1']:.3f}, "
      f"Acc@3: {val_metrics['acc3']:.3f}, "
      f"Acc@5: {val_metrics['acc5']:.3f}")

    if val_acc > best_val_acc:
        torch.save(model.state_dict(), "best_chess_model.pt")
        best_val_acc = val_acc
        print("Модель сохранена.")


EPOCH 1/5


Validation: 100%|██████████| 1/1 [00:05<00:00,  5.58s/it]


Train loss: 3.3241, acc: 0.1097
Validation: Loss: 15155.4805, Acc@1: 0.048, Acc@3: 0.123, Acc@5: 0.178
Модель сохранена.

EPOCH 2/5


Training:   0%|          | 0/18 [00:00<?, ?it/s]

: 

In [18]:
model.load_state_dict(torch.load("best_chess_model.pt", map_location=device))
model.eval()

fen = "rnb2rk1/pp3ppp/4p1n1/q1pP4/3P4/1QN1P1B1/PP3PPP/R3KB1R b KQ - 0 11"
top_moves = training.predict_move(model, fen, device, ALL_MOVES, MOVE_TO_INDEX, INDEX_TO_MOVE, top_k=5) #move, prob, all_probs, 
for i in top_moves:
    print(i)

{'топ': 1, 'ход': 'a5b6', 'вероятность': 0.23568940162658691}
{'топ': 2, 'ход': 'e6d5', 'вероятность': 0.0815194621682167}
{'топ': 3, 'ход': 'g6f4', 'вероятность': 0.06445614248514175}
{'топ': 4, 'ход': 'f7f5', 'вероятность': 0.048715658485889435}
{'топ': 5, 'ход': 'a5d8', 'вероятность': 0.04438405483961105}


In [ ]:
import inspect
print(inspect.signature(training.predict_move))

(model, fen_string, device, ALL_MOVES, MOVE_TO_INDEX, INDEX_TO_MOVE, top_k=5)
